# Data Accessibility Explorer

**Problem**: How can we make the mirrored Regulations.gov dataset easier to explore and analyze for researchers and non-technical stakeholders?

**Stakeholder Quotes:**
- "Accessing all the data in this format can be slow, but it's way better than Regulations.gov."
- "What can we build on top of the raw Mirrulations data?"

This notebook provides:
- Quick data exploration tools
- Schema documentation
- Sample queries for common tasks
- Export utilities

In [2]:
import duckdb
import pandas as pd

R2_BASE_URL = "https://pub-5fc11ad134984edf8d9af452dd1849d6.r2.dev"

conn = duckdb.connect()
conn.execute("INSTALL httpfs; LOAD httpfs;")
print("✓ DuckDB ready - querying Parquet files from R2")

✓ DuckDB ready - querying Parquet files from R2


## Data Overview

In [6]:
import os
import urllib.request

# Setup local data directory
DATA_DIR = "../data"
os.makedirs(DATA_DIR, exist_ok=True)

# Get dataset statistics
print("📊 Dataset Statistics")
print("=" * 50)

for data_type in ["dockets", "documents", "comments"]:
    local_path = f"{DATA_DIR}/{data_type}.parquet"
    url = f"{R2_BASE_URL}/{data_type}.parquet"
    
    # Download if not exists
    if not os.path.exists(local_path):
        print(f"  Downloading {data_type}.parquet...")
        req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urllib.request.urlopen(req) as response, open(local_path, 'wb') as out_file:
            out_file.write(response.read())
    
    # Get file size
    size_mb = os.path.getsize(local_path) / (1024 * 1024)
    
    stats = conn.execute(f"""
        SELECT 
            COUNT(*) as rows,
            COUNT(DISTINCT agency_code) as agencies
        FROM read_parquet('{local_path}')
    """).fetchone()
    print(f"  {data_type}: {stats[0]:,} rows, {stats[1]} agencies, {size_mb:.1f} MB")

📊 Dataset Statistics
  dockets: 346,173 rows, 194 agencies, 12.4 MB
  documents: 2,009,957 rows, 315 agencies, 54.2 MB
  comments: 24,774,674 rows, 178 agencies, 1969.5 MB


## Schema Reference

In [3]:
# Show schema for each dataset
for data_type in ["dockets", "documents", "comments"]:
    url = f"{R2_BASE_URL}/{data_type}.parquet"
    schema = conn.execute(f"DESCRIBE SELECT * FROM read_parquet('{url}')").fetchdf()
    print(f"\n📋 {data_type.upper()} Schema:")
    print("-" * 40)
    for _, row in schema.iterrows():
        print(f"  {row['column_name']}: {row['column_type']}")


📋 DOCKETS Schema:
----------------------------------------
  docket_id: VARCHAR
  agency_code: VARCHAR
  title: VARCHAR
  docket_type: VARCHAR
  modify_date: VARCHAR
  abstract: VARCHAR

📋 DOCUMENTS Schema:
----------------------------------------
  document_id: VARCHAR
  docket_id: VARCHAR
  agency_code: VARCHAR
  title: VARCHAR
  document_type: VARCHAR
  posted_date: VARCHAR
  modify_date: VARCHAR
  comment_start_date: VARCHAR
  comment_end_date: VARCHAR

📋 COMMENTS Schema:
----------------------------------------
  comment_id: VARCHAR
  docket_id: VARCHAR
  agency_code: VARCHAR
  title: VARCHAR
  comment: VARCHAR
  document_type: VARCHAR
  posted_date: VARCHAR
  modify_date: VARCHAR
  receive_date: VARCHAR


## Quick Search

In [4]:
def search_dockets(keyword, agency=None, limit=20):
    """Search dockets by keyword in title."""
    agency_filter = f"AND agency_code = '{agency}'" if agency else ""
    return conn.execute(f"""
        SELECT docket_id, agency_code, title, docket_type, modify_date
        FROM read_parquet('{R2_BASE_URL}/dockets.parquet')
        WHERE LOWER(title) LIKE '%{keyword.lower()}%'
        {agency_filter}
        ORDER BY modify_date DESC
        LIMIT {limit}
    """).fetchdf()

# Example: search for climate dockets
search_dockets("climate", limit=10)

,docket_id,agency_code,title,docket_type,modify_date
0,FAR-2021-0015,FAR,Federal Acquisition Regulation: Disclosure of ...,Rulemaking,2026-01-06T15:15:26Z
1,FAR-2021-0016,FAR,Federal Acquisition Regulation: Minimizing th...,Rulemaking,2026-01-06T15:15:14Z
2,EPA-HQ-OAR-2025-0162,EPA,Extension of Deadlines in Standards of Perform...,Rulemaking,2025-12-03T15:03:57Z
3,EPA-HQ-OAR-2024-0358,EPA,Reconsideration of Oil and Natural Gas Sector ...,Rulemaking,2025-10-10T17:05:28Z
4,DOE-HQ-2025-0207,DOE,A Critical Review of Impacts of Greenhouse Gas...,Rulemaking,2025-10-07T14:00:20Z
5,DOD-2025-OS-0342,DOD,Defense Organizational Climate Survey (DEOCS);...,Nonrulemaking,2025-07-29T12:47:52Z
6,EPA-HQ-OAR-2021-0317,EPA,"Standards of Performance for New, Reconstructe...",Rulemaking,2025-07-23T17:07:06Z
7,EPA-HQ-OAR-2023-0492,EPA,Transportation and Climate Division (TCD) Gran...,Nonrulemaking,2025-07-21T11:05:58Z
8,FAA-2025-0289,FAA,Climate Systems NW LLC - Exemption/Rulemaking,Nonrulemaking,2025-07-07T18:43:24Z
9,PTO-P-2025-0010,PTO,Termination of the Climate Change Mitigation P...,Nonrulemaking,2025-04-18T14:16:49Z


In [5]:
def get_docket_comments(docket_id, limit=100):
    """Get comments for a specific docket."""
    return conn.execute(f"""
        SELECT comment_id, title, posted_date, LEFT(comment, 200) as preview
        FROM read_parquet('{R2_BASE_URL}/comments.parquet')
        WHERE docket_id = '{docket_id}'
        ORDER BY posted_date DESC
        LIMIT {limit}
    """).fetchdf()

# Example
get_docket_comments("EPA-HQ-OAR-2021-0317", limit=5)

,comment_id,title,posted_date,preview
0,EPA-HQ-OAR-2021-0317-4068,Comment submitted by Interstate Natural Gas As...,2025-07-23T04:00:00Z,See Attached
1,EPA-HQ-OAR-2021-0317-4067,Supplemental Letter to Petition for Reconsider...,2024-10-23T04:00:00Z,See Attached
2,EPA-HQ-OAR-2021-0317-4066,Comment submitted by Air Quality Division (AQD...,2024-09-09T04:00:00Z,The Wyoming Department of Environmental Qualit...
3,EPA-HQ-OAR-2021-0317-4065,Comment submitted by American Petroleum Instit...,2024-09-04T04:00:00Z,The American Petroleum Institute (API) respect...
4,EPA-HQ-OAR-2021-0317-4063,"Comment submitted by Lumina Sky Consulting, PC",2024-08-28T04:00:00Z,Please see attached comments.


## Agency Explorer

In [6]:
# List all agencies with activity
agencies = conn.execute(f"""
    SELECT 
        agency_code,
        COUNT(*) as docket_count
    FROM read_parquet('{R2_BASE_URL}/dockets.parquet')
    GROUP BY agency_code
    ORDER BY docket_count DESC
""").fetchdf()

print(f"Found {len(agencies)} agencies with dockets:")
agencies.head(30)

Found 194 agencies with dockets:


,agency_code,docket_count
0,FDA,74303
1,FAA,58578
2,EPA,34093
3,DOT,14520
4,USCG,14055
5,FMCSA,10093
6,NOAA,9569
7,PHMSA,8522
8,NRC,6630
9,NHTSA,5376


## Export Utilities

In [7]:
def export_to_csv(query, filename):
    """Export query results to CSV."""
    df = conn.execute(query).fetchdf()
    df.to_csv(filename, index=False)
    print(f"Exported {len(df)} rows to {filename}")
    return df

# Example: export EPA dockets to CSV
# export_to_csv(f"""
#     SELECT * FROM read_parquet('{R2_BASE_URL}/dockets.parquet')
#     WHERE agency_code = 'EPA'
# """, "epa_dockets.csv")

In [8]:
def export_docket_summary(docket_id, output_dir="."):
    """Export a complete docket analysis package."""
    import os
    
    # Docket info
    docket = conn.execute(f"""
        SELECT * FROM read_parquet('{R2_BASE_URL}/dockets.parquet')
        WHERE docket_id = '{docket_id}'
    """).fetchdf()
    
    # Documents
    docs = conn.execute(f"""
        SELECT * FROM read_parquet('{R2_BASE_URL}/documents.parquet')
        WHERE docket_id = '{docket_id}'
    """).fetchdf()
    
    # Comments
    comments = conn.execute(f"""
        SELECT * FROM read_parquet('{R2_BASE_URL}/comments.parquet')
        WHERE docket_id = '{docket_id}'
    """).fetchdf()
    
    # Save
    docket.to_csv(f"{output_dir}/{docket_id}_docket.csv", index=False)
    docs.to_csv(f"{output_dir}/{docket_id}_documents.csv", index=False)
    comments.to_csv(f"{output_dir}/{docket_id}_comments.csv", index=False)
    
    print(f"Exported {docket_id}:")
    print(f"  - Docket info: 1 row")
    print(f"  - Documents: {len(docs)} rows")
    print(f"  - Comments: {len(comments)} rows")

# Example:
# export_docket_summary("EPA-HQ-OAR-2021-0317")

## Common Query Templates

In [9]:
print("""
📝 Common Query Templates
========================

# 1. Search dockets by keyword
SELECT * FROM read_parquet('{url}/dockets.parquet')
WHERE LOWER(title) LIKE '%keyword%'

# 2. Get comments for a docket
SELECT * FROM read_parquet('{url}/comments.parquet')
WHERE docket_id = 'AGENCY-YEAR-NNNN'

# 3. Find dockets with open comment periods
SELECT * FROM read_parquet('{url}/documents.parquet')
WHERE TRY_CAST(comment_end_date AS DATE) > CURRENT_DATE

# 4. Count comments by agency
SELECT agency_code, COUNT(*) FROM read_parquet('{url}/comments.parquet')
GROUP BY agency_code ORDER BY 2 DESC

# 5. Find duplicate comments
SELECT comment, COUNT(*) as cnt
FROM read_parquet('{url}/comments.parquet')
WHERE docket_id = 'XXX'
GROUP BY comment HAVING cnt > 1
""".format(url=R2_BASE_URL))


📝 Common Query Templates

# 1. Search dockets by keyword
SELECT * FROM read_parquet('https://pub-5fc11ad134984edf8d9af452dd1849d6.r2.dev/dockets.parquet')
WHERE LOWER(title) LIKE '%keyword%'

# 2. Get comments for a docket
SELECT * FROM read_parquet('https://pub-5fc11ad134984edf8d9af452dd1849d6.r2.dev/comments.parquet')
WHERE docket_id = 'AGENCY-YEAR-NNNN'

# 3. Find dockets with open comment periods
SELECT * FROM read_parquet('https://pub-5fc11ad134984edf8d9af452dd1849d6.r2.dev/documents.parquet')
WHERE TRY_CAST(comment_end_date AS DATE) > CURRENT_DATE

# 4. Count comments by agency
SELECT agency_code, COUNT(*) FROM read_parquet('https://pub-5fc11ad134984edf8d9af452dd1849d6.r2.dev/comments.parquet')
GROUP BY agency_code ORDER BY 2 DESC

# 5. Find duplicate comments
SELECT comment, COUNT(*) as cnt
FROM read_parquet('https://pub-5fc11ad134984edf8d9af452dd1849d6.r2.dev/comments.parquet')
WHERE docket_id = 'XXX'
GROUP BY comment HAVING cnt > 1

